In [1]:
%load_ext autoreload
%autoreload 2

import os
from travel import init_travel
init_travel()

In [5]:
 %reload_ext autoreload

# Test parsing method

In [18]:
import spacy
from travel.model.grounding import SpatialVisualFilter

nlp = spacy.load('en_core_web_md')
# test_questions = [
#     "Are the cherry tomatoes cut?",
#     "Are the cherry tomatoes not cut?",
#     "Are there cherry tomatoes in the bowl?",
#     "Are there any cherry tomatoes that are not in the bowl?"
# ]
test_questions = [
    "Is the garlic clove on the cutting board?",
    "Is the heat on?",
    "Is the heat on medium?",
    "Is the apple in a hand?",
    "Is the apple in someone's left hand?",
    "Is there tuna in the bowl that is not drained?",
    "Is the tablet on the table?",
    "Is the cleanser container somewhere not on the table?",
    "Is the wall painted?",
    "Is the container clean?",
    "Is there liquid in the container?"
]

SpatialVisualFilter.parse_questions_for_spatial_attention_filter(nlp, test_questions)

[(True, 'cutting board', 'Is the garlic clove in the image?'),
 (True, None, 'Is the heat on?'),
 (True, None, 'Is the heat on medium?'),
 (True, None, 'Is the apple in a hand?'),
 (True, None, "Is the apple in someone's left hand?"),
 (True, 'bowl', 'Is there tuna in the image that is not drained?'),
 (True, 'table', 'Is the tablet in the image?'),
 (False, 'table', 'Is the cleanser container somewhere in the image?'),
 (True, 'wall', 'Is the wall painted?'),
 (True, 'container', 'Is the container clean?'),
 (True, 'container', 'Is there liquid in the image?')]

# Test full spatial filter

In [16]:
from PIL import Image
import spacy
import torch
from travel.model.grounding import SpatialVisualFilter

visual_filter = SpatialVisualFilter(rephrase_questions=True, device="cuda:1" if torch.cuda.device_count() >= 2 else None)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [19]:
from pprint import pprint

nlp = spacy.load('en_core_web_md')

# questions = ["Are there cherry tomatoes in the bowl?", 
#              "Are there any cherry tomatoes that are not in the bowl?"]
# frames = [Image.open("demo_images/demo_frame2.png"),
#           Image.open("demo_images/demo_frame2.png")]

# questions = ["Is the knife dirty?"]
# frames = [Image.open("demo_images/demo_frame1.png")]

# questions = ["Is the plate clean?",
#              "Is the plate clean?"]
# frames = [Image.open("/nfs/turbo/coe-chaijy/sstorks/simulation_informed_pcr4nlu/TRAVEl/saved_results/vqa_mistake_detection/captaincook4d/VQG2VQA_captaincook4d_llava-1.5-7b-hf_20240617181122/VQA_cache_val_worker1of1/frames/frame_1_20_11_b4cf361c-3f3a-40aa-8ce8-b292073d741b.jpg"),
#           Image.open("/nfs/turbo/coe-chaijy/sstorks/simulation_informed_pcr4nlu/TRAVEl/saved_results/vqa_mistake_detection/captaincook4d/VQG2VQA_captaincook4d_llava-1.5-7b-hf_20240617181122/VQA_cache_val_worker1of1/frames/frame_9_19_3_15602923-6cff-466b-a6dc-0914d5473bd6.jpg")]

# questions = ["Is the tablet on the table?",
#             "Is the brick broken?"]
# frames = [Image.open('saved_results/vqa_mistake_detection/ego4d/VQG2VQA_ego4d_debug_llava-1.5-7b-hf_spatial1.0_20240624115037/VQA_cache_val_worker1of1/frames/frame_f6eb8e9b-a925-4112-9eec-32393192e435-39-easyneg_MisalignSRL_ARG1_c8f8ebf1-5613-4f4a-802c-ada5e4c4b651_127.0138486_51b16362-c1d0-481e-b521-6d8590ad4c92.jpg'),
#           Image.open("demo_images/demo_frameb.png").convert("RGB")]

questions = ["Is the container clean?",
             "Is there liquid in the container?"]
frames = [Image.open("demo_images/temp_container.png").convert("RGB"),
          Image.open("demo_images/temp_liquid_container.jpg").convert("RGB")]

new_frames, new_questions, target_object_counts = visual_filter(nlp, frames, questions)
pprint(target_object_counts)

running detection (cuda:0): 100%|██████████| 1/1 [00:00<00:00,  1.35it/s]


[(True, 'container', 'Is the container clean?'),
 (True, 'container', 'Is there liquid in the image?')]
[{'boxes': tensor([[751.4062,   5.3320, 904.2188, 146.4844],
        [829.2188, 152.9297, 930.4688, 259.4531],
        [840.0000, 279.1406, 957.1875, 464.2969]], device='cuda:0'),
  'labels': tensor([0, 0, 0], device='cuda:0'),
  'scores': tensor([0.4549, 0.3358, 0.3266], device='cuda:0')},
 {'boxes': tensor([[870.9375, 171.3281, 960.0000, 336.3281]], device='cuda:0'),
  'labels': tensor([0], device='cuda:0'),
  'scores': tensor([0.2857], device='cuda:0')}]
[['container'], ['container']]


running detection (cuda:0): 100%|██████████| 1/1 [00:00<00:00,  1.76it/s]

[{'boxes': tensor([[751.4062,   5.3320, 904.2188, 146.4844],
        [829.2188, 152.9297, 930.4688, 259.4531],
        [840.0000, 279.1406, 957.1875, 464.2969]], device='cuda:0'),
  'labels': tensor([0, 0, 0], device='cuda:0'),
  'scores': tensor([0.4549, 0.3358, 0.3266], device='cuda:0')},
 {'boxes': tensor([[870.9375, 171.3281, 960.0000, 336.3281]], device='cuda:0'),
  'labels': tensor([0], device='cuda:0'),
  'scores': tensor([0.2857], device='cuda:0')}]
[{0: 3}, {0: 1}]
[{'container': 3}, {'container': 1}]


## Show new frames

In [20]:
import matplotlib.pyplot as plt

# Assuming new_frames is a list of image data
fig, axarr = plt.subplots(1, len(new_frames), figsize=(22, 4))
fig.suptitle("TARGET FRAMES")

# Ensure axarr is always iterable
if len(new_frames) == 1:
    axarr = [axarr]

for frame, ax in zip(new_frames, axarr):
    if frame is not None:
        ax.imshow(frame)
        ax.axis('off')  # Hide the axes for better visualization

plt.tight_layout()
plt.show()

In [21]:
frame_saved = Image.open("/home/sstorks/coe-chaijy/sstorks/simulation_informed_pcr4nlu/TRAVEl/saved_results/vqa_mistake_detection/ego4d/VQG2VQA_ego4d_debug_llava-1.5-7b-hf_spatial1.0_20240624193456/VQA_cache_val_worker1of1/frames/frame_90093217-6a61-48ce-b737-e581499cf491-156-easyneg_MisalignSRL_V_ARG1_796bc463-2962-4a43-8a34-5f27983e0950_937.4757066666668_4aa57267-dad2-456e-bd3d-6781c41fd8d5.jpg")
plt.imshow(frame_saved)
plt.show()

## Show new questions

In [11]:
from pprint import pprint

pprint(new_questions)

['Is the container clean?']


# Test target object counting

In [5]:
from travel.model.grounding import TargetObjectCounterFilter
import spacy
from PIL import Image

filt = TargetObjectCounterFilter()
nlp = spacy.load('en_core_web_md')

questions = ["In a bowl, add the cut cherry tomatoes with a screwdriver", 
             "In a bowl, add the cut cherry tomatoes with a screwdriver",
             "In a bowl, add the cut cherry tomatoes with a screwdriver"]
frames = [Image.open("demo_images/demo_frame0.png"),
          Image.open("demo_images/demo_frame1.png"),
          Image.open("demo_images/demo_frame2.png")]

print(filt(nlp, frames, questions))

`low_cpu_mem_usage` was None, now set to True since model is quantized.
running detection (cuda:0): 100%|██████████| 1/1 [00:00<00:00,  1.26it/s]

[3, 20, 4]


In [2]:
from travel.model.grounding import TargetObjectCounterFilter, SpatialVisualFilter, ContrastiveRegionFilter
import spacy
from pprint import pprint

nlp = spacy.load('en_core_web_lg') # TODO: try lg model

questions = ["In a bowl, add the cut cherry tomatoes with a screwdriver",
             "Is the mechanics mobile seat being dragged?",
             "Is the mechanics mobile seat in someone's hands?",
             "Is the concrete cut?",
             "Is the lawn mower being held?",
             "Is the grass cut?",
             "Is there a hand on the branch?",
             "Is there any clay that is not in the brick mold?",
             "Is the concrete in the blue bucket?",
             "Is there a container in the image?"]

target_objects1 = TargetObjectCounterFilter.parse_sentences_for_target_objects(nlp, questions)
# target_objects2 = ContrastiveRegionFilter.parse_questions_for_contrastive_region_filter(nlp, questions)
spatial_parse = SpatialVisualFilter.parse_questions_for_spatial_attention_filter(nlp, questions)

for sentence, obj1, spatial in zip(questions, target_objects1, spatial_parse):
    print(f"{sentence} -> {obj1}")
    print(spatial)
    print("")

/home/sstorks/.cache/pypoetry/virtualenvs/travel-nQET-zRt-py3.10/lib/python3.10/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In a bowl, add the cut cherry tomatoes with a screwdriver -> ['bowl', 'cherry tomatoes', 'screwdriver']
(True, 'screwdriver', 'In a bowl, add the cut cherry tomatoes with a screwdriver')

Is the mechanics mobile seat being dragged? -> ['seat']
(True, 'mechanics', 'Is the mechanics mobile seat being dragged?')

Is the mechanics mobile seat in someone's hands? -> ['seat', 'hands']
(True, 'hands', 'Is the mechanics mobile seat in the image?')

Is the concrete cut? -> []
(True, None, 'Is the concrete cut?')

Is the lawn mower being held? -> ['lawn mower']
(True, 'lawn', 'Is the lawn mower being held?')

Is the grass cut? -> ['grass']
(True, 'grass', 'Is the grass cut?')

Is there a hand on the branch? -> ['hand', 'branch']
(True, 'branch', 'Is there a hand in the image?')

Is there any clay that is not in the brick mold? -> ['clay', 'brick mold']
(False, 'brick mold', 'Is there any clay that is in the image?')

Is the concrete in the blue bucket? -> ['concrete', 'bucket']
(True, 'bucket', 